In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import missingno as msno
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.metrics import mean_absolute_error ,r2_score ,mean_squared_error
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

In [16]:
test = pd.read_csv('/kaggle/input/jobathon-april-2022/test_6QvDdzb.csv')

In [3]:
train = pd.read_csv('/kaggle/input/jobathon-april-2022/train_E1GspfA.csv')

sample = pd.read_csv('/kaggle/input/jobathon-april-2022/sample_4E0BhPN.csv')

In [4]:
def convert_date_new(df,feature):
    df[feature]=pd.to_datetime(df[feature])
    df['year']= df[feature].dt.year
    df['month']= df[feature].dt.month
    df['day']= df[feature].dt.day
    df['weekday']= df[feature].dt.weekday
    
def outlier_removal(df,feature):
    df = df[np.abs(df[feature]-df[feature].mean())<=(3*df[feature].std())]
    
def train_model(model,df):
    X = df.drop(['demand'],axis=1)
    y = df['demand']
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.25, random_state=42)
    model.fit(X_train,y_train)
    return pred_error(model,X_test,y_test)
    
def pred_error(model,test,old):
    new = model.predict(test)
    rmse = (np.sqrt(mean_squared_error(old, new)))
    r2 = r2_score(old, new)
    MEA=mean_absolute_error(old, new)
    print('Testing performance')
    print('RMSE: {:.2f}'.format(rmse))
    print('R2: {:.2f}'.format(r2))
    print('MEA: {:.2f}'.format(MEA))
    
def pred_submission(model,test):
    y_output = model.predict(test)
    sample['demand']=y_output
    return sample.to_csv('submission1.csv',index=False)

def drop_col(df,col):
    return df.drop([col], axis=1)

In [5]:
# model
# ctb = CatBoostRegressor(iterations=150,l2_leaf_reg=0.2,max_depth=8,learning_rate=0.1,verbose=0)

In [5]:
convert_date_new(train,'date')

df_o_d=train.copy(deep=True)#df

In [17]:
convert_date_new(test,'date')

## Before Removing Date Column

In [6]:
# # CatBoostRegressor(iterations=150,l2_leaf_reg=0.2,max_depth=8,learning_rate=0.1,verbose=0)
# convert_date_new(train,'date')
# convert_date_new(test,'date')
# outlier_removal(train,'demand')
# train_model(ctb,train)

Testing performance
RMSE: 32.12
R2: 0.38
MEA: 27.02

## After Removing Date Column

In [7]:
# # CatBoostRegressor(iterations=150,l2_leaf_reg=0.2,max_depth=9,learning_rate=0.1,verbose=0)
# convert_date_new(train,'date')
# convert_date_new(test,'date')
# outlier_removal(train,'demand')
# train_n=train.copy(deep='True')
# train_n=train_n.drop(['date'], axis=1)
# train_model(ctb,train_n)

Testing performance
RMSE: 32.30
R2: 0.38
MEA: 27.21

# submission file with date

In [8]:
# ctb = CatBoostRegressor(iterations=150,l2_leaf_reg=0.2,max_depth=8,learning_rate=0.1,verbose=0)
# convert_date_new(train,'date')
# convert_date_new(test,'date')
# outlier_removal(train,'demand')
# train_model(ctb,train)
# pred_submission(ctb,test)

Testing performance
RMSE: 32.12
R2: 0.38
MEA: 27.02

In [6]:
import catboost as cb

# With Outlier & Date Column

In [7]:
# from sklearn.model_selection import train_test_split

# X = df_o_d.drop(['demand'],axis=1)
# y = df_o_d['demand']
# X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
# train_dataset = cb.Pool(X_train, y_train) 
# test_dataset = cb.Pool(X_test, y_test)

# from catboost import CatBoostRegressor
# model = cb.CatBoostRegressor(loss_function='RMSE',verbose=0,task_type="GPU",
#                            devices='0:1')
# grid = {'iterations': [100, 150, 200,300],
#         'learning_rate': [0.03,0.05, 0.1],
#         'depth': [2, 4, 6, 7,8,9],
#         'l2_leaf_reg': [0.05,0.1,0.2, 0.5, 1,2, 3]}
# model.grid_search(grid, train_dataset)
# pred = model.predict(X_test)
# rmse = (np.sqrt(mean_squared_error(y_test, pred)))
# r2 = r2_score(y_test, pred)
# print('Testing performance')
# print('RMSE: {:.2f}'.format(rmse))
# print('R2: {:.2f}'.format(r2))

Training on fold [1/3]
bestTest = 32.3581226
bestIteration = 299
Training on fold [2/3]
bestTest = 33.43154897
bestIteration = 289
Testing performance
RMSE: 32.46
R2: 0.39

# Date column and Without Outlier 

In [8]:
out = df_o_d[np.abs(df_o_d["demand"]-df_o_d["demand"].mean())<=(3.5*df_o_d["demand"].std())]
X = out.drop(['demand'],axis=1)
y = out['demand']
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
train_dataset = cb.Pool(X_train, y_train) 
test_dataset = cb.Pool(X_test, y_test)

model = cb.CatBoostRegressor(loss_function='RMSE',verbose=0,task_type="GPU",
                           devices='0:1')
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
model.grid_search(grid, train_dataset)


Training on fold [2/3]
bestTest = 32.4857845
bestIteration = 199

'params': {'depth': 8,
  'l2_leaf_reg': 1,
  'iterations': 200,
  'learning_rate': 0.1}

In [ ]:
# pred = model.predict(X_test)
# rmse = (np.sqrt(mean_squared_error(y_test, pred)))
# r2 = r2_score(y_test, pred)
# print('Testing performance')
# print('RMSE: {:.2f}'.format(rmse))
# print('R2: {:.2f}'.format(r2))

# Without date & outlier

In [10]:
out_d = out.drop(['date'], axis=1)
X = out_d.drop(['demand'],axis=1)
y = out_d['demand']
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
train_dataset = cb.Pool(X_train, y_train) 
test_dataset = cb.Pool(X_test, y_test)

model = cb.CatBoostRegressor(loss_function='RMSE',verbose=0,task_type="GPU",
                           devices='0:1')
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
model.grid_search(grid, train_dataset)
# pred = model.predict(X_test)
# rmse = (np.sqrt(mean_squared_error(y_test, pred)))
# r2 = r2_score(y_test, pred)
# print('Testing performance')
# print('RMSE: {:.2f}'.format(rmse))
# print('R2: {:.2f}'.format(r2))

bestTest = 32.90719586
bestIteration = 196
Training on fold [1/3]
bestTest = 33.00886845
bestIteration = 183
Training on fold [2/3]
bestTest = 32.43745959
bestIteration = 182
{'params': {'depth': 8,
  'iterations': 200,
  'learning_rate': 0.1,
  'l2_leaf_reg': 0.5},

In [12]:
test = test.drop(['date'], axis=1)
train = train[np.abs(df_o_d["demand"]-df_o_d["demand"].mean())<=(3.5*df_o_d["demand"].std())]
train = train.drop(['date'], axis=1)
X = train.drop(['demand'],axis=1)
y = train['demand']

In [18]:
ctb = CatBoostRegressor(iterations=200,l2_leaf_reg=0.5,
                    max_depth=8,
                    learning_rate=0.1,verbose=0)
ctb.fit(X,y)
y_output=ctb.predict(test)
y_output

In [19]:
sample['demand']=y_output
sample.to_csv('submission1.csv',index=False)